<nav class="navbar navbar-default">
  <div class="container-fluid">
    <div class="navbar-header">
      <a class="navbar-brand" href="0_Dataoving3.ipynb">Dataøving 3</a>
    </div>
    <ul class="nav navbar-nav">
        <li><a href="Oppgave1.ipynb">Oppgave 1 - Introduksjon til spektral lekkasje</a></li>
        <li><a href="Oppgave2.ipynb">Oppgave 2 - Zero Padding og DTFT</a></li>
        <li><a href="Oppgave3.ipynb">Oppgave 3 - Vindusfunksjoner</a></li>
        <li class="active"><a href="Oppgave4.ipynb">Oppgave 4 - Spektrogram: <i>Tilstandsovervåking av maskineri del 2</i></a></li>
    </ul>
  </div>
</nav>

# Spektrogram

**Tema:**
* Spektrogram

**Læringsmål:**
* Erfare hvordan vinduslengde $N$ påvirker frekvensanalyse
* Erfare hvordan vindusfunksjoner påvirker frekvensanalyes
* Erfare bruk av zero-padding i praksis

**Bibliotek og notebook-konfigurasjon:**

In [1]:
from numpy import sin, cos, pi, exp, empty, mean, absolute, angle # Sentrale matematiske funksjoner
from numpy.fft import fft, ifft             # DFT og IDFT
import numpy as np                          # Importer funksjonalitet fra numpy biblioteket med prefiks "np"
import matplotlib.pyplot as plt             # Importer pyplot modulen i matplotlib med prefiks "plt"
import scipy.signal.windows as win          # Importerer funksjonalitet for å generere en rekke vindusfunksjoner
from scipy.io import loadmat, savemat, wavfile               # Lesing av data fra '.mat' fil
from scipy.signal import spectrogram        # Lar oss generere et spektrogram som analyserer et signal i tid og frekvens

%matplotlib ipympl

# Problemstilling: Tilstandsovervåking i maskineri del 2

I dataøving 2 utnyttet vi frekvensforskyvningsegenskapene ved frekvensmiksing av signal til å avgjøre tilstedeværelsen av en spesiell frekvenskomponent i et signal. Signalet var hentet fra en vibrasjonssensor som målte vibrasjoner i kulelageret på et kontaktpunkt i en vindmølle, der kraftige vibrasjoner med én spesifikk frekvens ville indikere at en kule i kulelageret var skadet.

Vi skal nå se på et lignende scenario der en drivaksling overfører kraften fra en elektrisk motor til propellen på en båt. En vibrasjonssensor brukes til å plukke opp vibrasjoner i et kulelager koblet til akslingen, der akslingens radius er 3.1 ganger så stor som kulenes radius. 

![Illustrasjon av kulelager med skade](Figurer/Kulelager.png)

Et slikt system vil under normale omstendigheter produsere vibrasjoner der frekvensinnholdet er komponert av overtoner til rotasjonshastigneten. Dette betyr at dersom akslingen har rotasjonshastighet $\rho$ omdreininger per sekund, vil frekvensanalyse av signalet fra vibrasjonssensoren ha tydelige toppunkt der er et heltalls multiplum av rotasjonshastigheten $f \in \{ 1\cdot \rho,\ 2\cdot \rho,\ 3\cdot \rho,\ \ldots\}$.

Dersom det skulle oppstå skade på en av kulene i kulelageret, vil de produserte vibrasjonene *i tillegg* inneholde frekvenskomponenter som er overtoner av kulens rotasjonshastighet, altså $3.1\cdot \rho$. Frekvensanalyse av signalet fra vibrasjonssensoren burde dermed ha tydelige toppunkt i frekvensene $f \in \{ 3.1\cdot \rho,\ 6.2 \cdot \rho,\ 9.3\cdot \rho,\ \ldots \}$ dersom det er en skade på en av kulene. Figuren nedenfor viser hvordan frekvensinnholdet for et slikt signal fra et skadet kulelager kan se ut dersom rotasjonshastigheten $\rho = 10$ omdreininger per sekund.


<img src="Figurer/Oppg4_skadeeksempel.png" style="width: 80%" />


Vi ser ut ifra figuren at de områdene i frekvensspekteret som er av interesse for å detektere en skade i kulelageret ligger tett inntil de områdene der vi forventer at de normale vibrasjonene vil gi utslag. Her er det med andre ord viktig at frekvensanalysen er basert på stor nok vinduslengde $N$ til at vi klarer å skille de normale frekvenskomponentene fra de som vil indikere en skade. 

Dessverre er det enda en detalj i dette systemet som kan skape problemer når vibrasjonssignalet skal analyseres. Akslingen har nemlig ikke konstant rotasjonshastighet, men vil variere hastigheten over tid. Dette gir oss noe nytt å ta hensyn til under frekvensanalyse ettersom frekvensanalyse av $N$ signalsampler viser **alt** frekvenssinnhold i en tidsperiode med varighet $\Delta t = N\cdot T_s$, og frekvensinnholdet til signalet varierer over tid, betyr dette at vi vil plukke opp flere frekvenser desto lengre vinduslengde $N$ vi bruker. Sagt på en annen måte: for å kunne fastsette frekvensen til et rent sinussignal med høy presisjon, må det aktuelle sinussignalet observeres over lang tid. Dette er godt et eksempel på [uskarphetsrelasjon](https://en.wikipedia.org/wiki/Uncertainty_principle#Harmonic_analysis), som ellers ofte forbindes med kvantemekanikk. [Her](https://www.youtube.com/watch?v=MBnnXbOM5S4) er en interresant video som forklarer relevansen for uskarphetsprinsippet i signalbehandling på en god måte. Dersom du ønsker å teste ut prinsippet selv, kan du bruke `SignalAnalyzer()`-verktøyet til å utføre frekvensanalyse med varierende vinduslengde på et sinussignal med gradvis økende frekvens [her](Signalanalyse%20av%20chirp-signal.ipynb).


Audiofilen `Oppgave4.wav` inneholder en logg av vibrasjonssignalet fra et kulelager som gikk i stykker. Du kan laste inn signalsamplene og samplingsfrekvensen ved å kjøre kodecellen nedenfor. Vi skal nå prøve å kalibrere de ulike parametrene for frekvensanalysen slik at vi klarer å identifisere tilstedeværelsen av en skade på kulelageret så tidlig som mulig. For å kunne analysere frekvensinnholdet til et signal over tid skal vi ta i bruk et nytt analyseverktøy; ***spektrogrammet***.

In [2]:
fs, data = wavfile.read(r'Datafiler/Oppgave4.wav') 
xn = data/0x8000

### Spektrogram

Et spektrogram er en visuell framstilling av frekvensinnholdet til et signal over tid. 

1. Del opp signalet i mindre segment med samme lengde på $N$ sampler.
    * For å øke antallet datapunkt på tidsaksen er det ofte hensiktsmessig at disse segmentene overlapper hverandre.
2. Dersom ønskelig, ufør vekting av alle signalsegmentene med en *vindusfunksjon* for å motvirke spektral lekkasje.
3. Dersom ønskelig, bruk Zero Padding til å øke antallet datapunkt i frekvensaksen.
4. Regn ut DFT av hvert individuelle signalsegment.
5. Kombiner alle de utregnede DFT-sekvensene til å lage et slags terreng-kart for signalet, med frekvens langs én akse og tid langs den andre. 
    * Enkleste måte å presentere dataene er å vise frekvenskomponent-amplitude med fargetone.

I Python kan vi lage et spektrogram av et signal `xn` med samplingsfrekvens `fs` ved bruk av funksjonen `spectrogram(xn, fs)` fra modulen `scipy.signal`. Denne funksjonen vil gi oss en samling med arrays som vi kan lage en grafisk fremstilling av ved å brukde pyplot-funksjonen `pcolormesh()`. Når vi lager spektrogrammet er det en rekke parametre vi kan angi utover signalsamplene `xn` og samplingsfrekvensen `fs` ved å legge til nøkkelord-argument i funksjonskallet. Disse er forklart i tabellene nedenfor:

#### Aktuelle parametre å justere ved utregning av spektrogram:

|Argument-navn | Beskrivelse|
|:--|:--|
| `window` | Spesifiserer vindusfunksjonen, det enkleste er å angi navnet på vindusfunksjonen som tekststreng. En liste med støttede vindusfunksjoner finner du [her](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.get_window.html).|
|`nperseg` | Spesifiserer vinduslengden i antall sampler $N$.|
|`noverlap` | Spesifiserer antall sampler overlapping mellom etterfølgende utklipp. Må være mindre enn vinduslengden $N$|
|`nfft`| Spesifiserer antallet punkter som utregnes i DFT. Zero Padding brukes for å oppnå flere DFT-element enn vinduslengden $N$. Må være større enn eller lik vinduslengden $N$.

#### Forklaring av returverdier:
|Returverdi | Forklaring|
|:--|:--|
|`f_axis`| En 1-D `ndarray` med alle frekvensverdiene som korresponderer med elementene i samtlige DFT-sekvenser.
|`t_axis`| En 1-D `ndarray` med startidspunktet for alle signalvinduene som har blitt utregnet DFT av.
|`Sxx`| En 2-D `ndarray` der `Sxx[i][j]` tilsvarer effekten $P$ til signalkomponenten med frekvens $f=$ `f_axis[j]` ved tidspunkt $t=$ `t_axis[i]`.
 
#### Aktuelle parametre å justere for grafisk framstilling:
| Argument-navn | Beskrivelse |
|:--|:--|
|`v_min`| Minimumsverdi i fargeskalaen. Alle verdier mindre enn `v_min` visualiseres med mørkeblå fargetone.
|`v_max`| Maksimumsverdi i fargeskalaen. Alle verdier større enn `v_max` visualiseres med gul fargetone.

Til slutt kan det være aktuelt å bruke pyplot-funksjonene `ylim()` og `xlim()` til å fokusere på utvalgte områder på spektrogrammet.

## Oppgave:

### a)
Kodecellen nedenfor produserer et spektrogram som viser frekvensinnholdet til signalloggen `xn`, og visualiserer det med et fargekart. Dessverre er det ikke så lett å plukke ut de frekvenskomponentene som ville indikere tilstedeværelsen av en skadet kule fra de andre komponentene i signalet. 

Bruk det du har lært i oppgave 1-3 til å justere på parametrene i spektrogrammet slik at resultatet blir et spektrogram hvor de individuelle frekvenskomponentene i størst mulig grad enkle å identifisere, selv om de kanskje ligger nære hverandre.

In [4]:
f_axis, t_axis, Sxx = spectrogram(xn,                # Fullt signal
                                  fs/2,              # Samplingsfrekvens
                                  window = 'hamming', # Vindusfunksjon (rektangulær)
                                  nperseg = 1024,      # Vinduslengde
                                  noverlap = 512,      # Antall sampler overlapping
                                  nfft =2048,          # Oppløsning på DFT etter zero padding
                                  )

plt.close(1);plt.figure(1, figsize=(10,6))

plt.pcolormesh(t_axis,            # Array med tidsstempel. Produsert av spectrogram()
               f_axis,            # Array med frekvenser. Produsert av spectrogram()
               10*np.log10(Sxx),  # Konvertering av spektrogrammatrise til logaritmisk skala
               vmax =-20,          # Høyeste dB-verdi på fargekartet
               vmin = -60,        # Laveste dB-verdi på fargekartet
               shading='auto'
               )
plt.colorbar(aspect=50, label=r'Spectral Density (dB/Hz)')

#plt.ylim([0, 120]) # Du kan justere på grensene i y-aksen dersom du vil inspisere en spesiell del av plottet
#plt.xlim([0, 600]) # Du kan justere på grensene i x-aksen dersom du vil inspisere en spesiell del av plottet
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.tight_layout(pad=0.1, w_pad=1.0, h_pad=1.0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### b) 

Kommenter på hvordan ulike verdier for  vinduslengde, vindusfunksjon, zero padding og overlapping ser ut til å påvirke spektrogrammet. Hva er det første tidspunktet der du ser antydning til et skadesignal?

#### Vinduslengde
Vinduslengde $N$ øker frekvensoppløsningen i y-aksen, og gir dermed et større antall "piksler" i y-retning. Vinduslengden må være større for å kunne skille mellom to relativt nærliggende frekveskomponenter.

#### Vindusfunksjon
Vindusfunksjonen "boxcar" (rektangulørt vindu) resulterer i tydelig spektral lekkasje. For å kunne fremstille frekvenskomponentene som mer konsentrerte linjer, vil en annen vindusfunksjon som  f.eks. Hamming være fordelaktig.

#### Zero Padding
Ved bruk av zero padding kan man øke frekvensoppløsningen (og dermed redusere oppløsningsbåndbredden) i fouriertransformene, uten å måtte anvende større utklipp av signalet. Dette vil "fylle ut" flere punkter langs en kontinuerlig kurve, og kan resultere i et mindre hakkete spektrogram i y-retning.

#### Overlapping 
Ved å ta i bruk overlappende segment kan vi øke oppløsningen i tidsaksen, og dermed få flere "piksler" i y-retning uten at det går på bekostning av frekvensoppløsningen.

Med justering av disse parametrene, er det mulig å produsere et spektrogram der det er mulig å se en antydning til et skadesignal på $3.1\cdot \rho$ rundt $t = 50$ sekund.